In [9]:
import geopandas as gpd
from shapely.geometry import LineString, Point
from shapely.ops import split, unary_union
from shapely.geometry.polygon import orient
import numpy as np
import rasterio
from rasterio.features import rasterize
from skimage.morphology import skeletonize
from geoprocessing_tools import multipolygon_to_polygon

def create_centerline(polygon_file, output_CL_file, tolerance=50):
    """Process each polygon in the GeoPackage to create centerlines, apply smoothing, and save them."""

    def polygon_to_raster(poly, cell_size=1):
        """Convert a polygon to a raster array."""
        bounds = poly.bounds
        width = int(np.ceil((bounds[2] - bounds[0]) / cell_size))
        height = int(np.ceil((bounds[3] - bounds[1]) / cell_size))
        transform = rasterio.transform.from_origin(bounds[0], bounds[3], cell_size, cell_size)
        raster = rasterize([(poly, 1)], out_shape=(height, width), transform=transform)
        return raster, transform

    def raster_to_centerline(raster, transform):
        """Convert raster array to a centerline geometry."""
        skeleton = skeletonize(raster == 1)
        points = [Point(*rasterio.transform.xy(transform, row, col, offset='center'))
                  for row in range(skeleton.shape[0]) for col in range(skeleton.shape[1]) if skeleton[row, col]]
        if points:
            line = LineString(points)
            return line
        return None

    def smooth_line(line, tolerance):
        """Smooth the line geometry using the Douglas-Peucker algorithm."""
        if line:
            return line.simplify(tolerance, preserve_topology=False)
        return line

    def calc_centerline(polygon, cell_size=1):
        """Main function to create and smooth centerline from a polygon."""
        raster, transform = polygon_to_raster(polygon, cell_size)
        centerline = raster_to_centerline(raster, transform)
        smoothed_centerline = smooth_line(centerline, tolerance)
        return smoothed_centerline

    gdf = gpd.read_file(polygon_file)
    gdf['centerline'] = gdf['geometry'].apply(lambda x: calc_centerline(x))

    # Remove entries where no centerline was found
    centerlines_gdf = gdf.dropna(subset=['centerline'])
    centerlines_gdf = centerlines_gdf.set_geometry('centerline', drop=True)  # Set 'centerline' as the geometry column and drop the old one
    centerlines_gdf.crs = gdf.crs  # Ensure CRS is preserved
    # Save to a new GeoPackage
    centerlines_gdf.to_file(output_CL_file, driver='GPKG')

    return centerlines_gdf


def create_perpendicular_lines(gpkg_path, distance=100, spacing=1):
    # Load the centerline from the geopackage
    gdf = gpd.read_file(gpkg_path)
    
    # Initialize an empty list to store perpendicular lines
    perpendiculars = []
    
    # Iterate through each feature in the GeoDataFrame
    for _, row in gdf.iterrows():
        geometry = row['geometry']
        
        # Handle MultiLineString appropriately using `geoms`
        if isinstance(geometry, MultiLineString):
            line_parts = geometry.geoms
        else:
            line_parts = [geometry]

        # Process each line part
        for line in line_parts:
            coords = np.array(line.coords)
            for i in range(0, len(coords) - 1, spacing):  # Adjust spacing here
                p1, p2 = coords[i], coords[i+1]
                dx, dy = p2[0] - p1[0], p2[1] - p1[1]
                
                # Calculate the perpendicular vector
                len_vector = np.sqrt(dx**2 + dy**2)
                perp_vector = (-dy, dx)
                
                # Normalize and scale the vector
                perp_vector = (perp_vector[0] / len_vector * distance, perp_vector[1] / len_vector * distance)
                
                # Calculate mid-point of the line segment
                mid_point = ((p1[0] + p2[0]) / 2, (p1[1] + p2[1]) / 2)
                
                # Create the perpendicular line segment
                perp_line = LineString([
                    (mid_point[0] + perp_vector[0], mid_point[1] + perp_vector[1]),
                    (mid_point[0] - perp_vector[0], mid_point[1] - perp_vector[1])
                ])
                
                # Append the perpendicular line to the list
                perpendiculars.append({'geometry': perp_line})
    
    # Convert list to GeoDataFrame
    perpendiculars_gdf = gpd.GeoDataFrame(perpendiculars, crs=gdf.crs)
    
    # Save the perpendicular lines to the same geopackage
    out_gpkg_path = gpkg_path.replace('.gpkg', '_perpendiculars.gpkg')
    perpendiculars_gdf.to_file(out_gpkg_path, driver='GPKG')

def create_smooth_perpendicular_lines(gpkg_path, line_length=60, spacing=5, window=20):
    # Load the centerline from the geopackage
    gdf = gpd.read_file(gpkg_path)
    
    # Initialize an empty list to store perpendicular lines
    perpendiculars = []
    
    # Iterate through each feature in the GeoDataFrame
    for _, row in gdf.iterrows():
        geometry = row['geometry']
        
        # Handle MultiLineString appropriately using `geoms`
        if isinstance(geometry, MultiLineString):
            line_parts = geometry.geoms
        else:
            line_parts = [geometry]

        # Process each line part
        for line in line_parts:
            length = line.length
            num_samples = int(np.floor(length / spacing))
            for i in range(num_samples + 1):
                # Calculate the point at each meter
                point = line.interpolate(i * spacing)
                
                # Get points 20 meters ahead and behind
                point_back = line.interpolate(max(0, i * spacing - window))
                point_forward = line.interpolate(min(length, i * spacing + window))
                
                # Calculate vectors to these points
                dx_back, dy_back = point.x - point_back.x, point.y - point_back.y
                dx_forward, dy_forward = point_forward.x - point.x, point_forward.y - point.y
                
                # Average the vectors
                dx_avg = (dx_back + dx_forward) / 2
                dy_avg = (dy_back + dy_forward) / 2
                
                # Calculate the perpendicular vector
                len_vector = np.sqrt(dx_avg**2 + dy_avg**2)
                perp_vector = (-dy_avg, dx_avg)
                
                # Normalize and scale the vector
                perp_vector = (perp_vector[0] / len_vector * line_length, perp_vector[1] / len_vector * line_length)
                
                # Create the perpendicular line segment
                perp_line = LineString([
                    (point.x + perp_vector[0], point.y + perp_vector[1]),
                    (point.x - perp_vector[0], point.y - perp_vector[1])
                ])
                
                # Append the perpendicular line to the list
                perpendiculars.append({'geometry': perp_line})
    
    # Convert list to GeoDataFrame
    perpendiculars_gdf = gpd.GeoDataFrame(perpendiculars, crs=gdf.crs)
    
    # Save the perpendicular lines to the same geopackage
    out_gpkg_path = gpkg_path.replace('.gpkg', '_perpendiculars_100m.gpkg')
    perpendiculars_gdf.to_file(out_gpkg_path, driver='GPKG')

def segment_stream_polygon(stream_polygon_path, centerline_path, output_path, n_segments = 200, window=20):
    # Load the shapefile and the centerline
    gdf = gpd.read_file(stream_polygon_path)
    centerline_gdf = gpd.read_file(centerline_path)
    
    # Assuming the polygon to segment is the first feature in the shapefile
    polygon = gdf.geometry[0]
    centerline = centerline_gdf.geometry[0]
    
    # Calculate interval along the centerline to place cutting points
    line_length = centerline.length
    interval = line_length / n_segments
    
    # Initialize list to store cutting lines
    cutting_lines = []
    
    for i in range(1, n_segments):
        # Calculate the primary interpolation point
        point = centerline.interpolate(i * interval)

        # Calculate points 20 meters behind and ahead for rolling average
        point_back = centerline.interpolate(max(0, i * interval - window))
        point_forward = centerline.interpolate(min(line_length, i * interval + window))
        
        # Determine vectors to these points
        dx_back, dy_back = point.x - point_back.x, point.y - point_back.y
        dx_forward, dy_forward = point_forward.x - point.x, point_forward.y - point.y
        
        # Average the vectors
        dx_avg = (dx_back + dx_forward) / 2
        dy_avg = (dy_back + dy_forward) / 2
        
        # Compute the perpendicular vector
        length_vector = np.sqrt(dx_avg**2 + dy_avg**2)
        perp_dx = -dy_avg / length_vector
        perp_dy = dx_avg / length_vector
        
        # Define a long perpendicular line for cutting
        start_point = Point(point.x + perp_dx * 1000, point.y + perp_dy * 1000)
        end_point = Point(point.x - perp_dx * 1000, point.y - perp_dy * 1000)
        cutting_lines.append(LineString([start_point, end_point]))
    
    # Initial set of segments
    segments = [polygon]

    # Split the polygon with each line
    for line in cutting_lines:
        new_segments = []
        for segment in segments:
            split_result = split(segment, line)
            new_segments.extend(split_result.geoms)
        segments = new_segments

    # Convert segments to GeoDataFrame
    segment_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(segments))

    # Set the same CRS as the original
    segment_gdf.crs = gdf.crs

    # Save to a new shapefile
    segment_gdf.to_file(output_path)
    

In [2]:
import geopandas as gpd

def rank_elevation_mean(input_geopackage, output_geopackage):
    """
    Reads a GeoPackage, ranks features based on the 'Elevation Mean' field,
    adds these ranks as a new field 'id', and writes the result to a new GeoPackage.

    Parameters:
    input_geopackage (str): Path to the input GeoPackage.
    output_geopackage (str): Path to the output GeoPackage where the result will be saved.
    """
    # Load the geopackage into a GeoDataFrame
    gdf = gpd.read_file(input_geopackage)

    # Check if 'Elevation Mean' column exists
    if 'Elevation Mean' not in gdf.columns:
        raise ValueError("The column 'Elevation Mean' does not exist in the provided GeoPackage.")

    # Rank the 'Elevation Mean' values from smallest to largest
    # Method should be one of 'average', 'min', 'max', 'first', 'dense'
    gdf['fid'] = gdf['Elevation Mean'].rank(method='first').astype(int)

    # Write the modified GeoDataFrame to a new geopackage
    gdf.to_file(output_geopackage, driver='GPKG')





In [10]:
from compute_hillslope_attributes import aggregate_raster_stats
import os

channel_poly_dir = r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Watershed Stats\Channels\Channel Polygons JTM"
centerline_dir = r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Watershed Stats\Channels\Centerlines"
output_single_poly_dir = r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Watershed Stats\Channels\One Part Polygons"
output_segment_poly_dir = r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Watershed Stats\Channels\Segmented Polygons"


#channel_poly_paths = [os.path.join(channel_poly_dir, f) for f in os.listdir(channel_poly_dir) if f.endswith('.gpkg')]
#centerline_paths = [os.path.join(centerline_dir, f) for f in os.listdir(centerline_dir) if f.endswith('.gpkg')]

channel_poly_paths = [
    r"Y:\ATD\GIS\Bennett\Channel Polygons\ME_Channel.gpkg",
r"Y:\ATD\GIS\Bennett\Channel Polygons\MM_Channel.gpkg",
r"Y:\ATD\GIS\Bennett\Channel Polygons\MW_Channel.gpkg",
r"Y:\ATD\GIS\Bennett\Channel Polygons\UE_Channel.gpkg",
r"Y:\ATD\GIS\Bennett\Channel Polygons\UM_Channel.gpkg",
r"Y:\ATD\GIS\Bennett\Channel Polygons\UW_Channel.gpkg",
]

centerline_paths = [
    r"Y:\ATD\GIS\Bennett\Channel Polygons\Centerlines\ME_centerline.gpkg",
r"Y:\ATD\GIS\Bennett\Channel Polygons\Centerlines\MM_centerline.gpkg",
r"Y:\ATD\GIS\Bennett\Channel Polygons\Centerlines\MW_centerline.gpkg",
r"Y:\ATD\GIS\Bennett\Channel Polygons\Centerlines\UE_centerline.gpkg",
r"Y:\ATD\GIS\Bennett\Channel Polygons\Centerlines\UM_centerline.gpkg",
r"Y:\ATD\GIS\Bennett\Channel Polygons\Centerlines\UW_centerline.gpkg",
]

output_rough_CL_dir = r"Y:\ATD\GIS\Bennett\Channel Polygons\Centerlines"

if not os.path.exists(output_single_poly_dir):
    os.makedirs(output_single_poly_dir)
if not os.path.exists(output_segment_poly_dir):
    os.makedirs(output_segment_poly_dir)
if not os.path.exists(output_rough_CL_dir):
    os.makedirs(output_rough_CL_dir)

DEM_path = r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Terrain Feature Rasters\Slope.tif"
#
segmented_poly_dir = r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Watershed Stats\Channels\Segmented Polygons"
segmented_poly_paths = [os.path.join(segmented_poly_dir, f) for f in os.listdir(segmented_poly_dir) if f.endswith('.gpkg')]

for channel_path, centerline_path in zip(channel_poly_paths, centerline_paths):
    single_poly_name = os.path.basename(channel_path)[0:3] + "_channel_single_poly.gpkg"
    output_single_poly_path = os.path.join(output_single_poly_dir, single_poly_name)
    # output_segment_name = os.path.basename(channel_path)[0:3] + "_channel_segmented.gpkg"
    # output_segment_poly_path = os.path.join(output_segment_poly_dir, output_segment_name)
    output_CL_file = os.path.join(output_rough_CL_dir, "Rough " + os.path.basename(centerline_path))
    
    multipolygon_to_polygon(channel_path, output_single_poly_path)
    create_centerline(output_single_poly_path, output_CL_file, tolerance=300)

    #segment_stream_polygon(output_single_poly_path, centerline_path, output_segment_poly_path)
# for output_segment_poly_path in segmented_poly_paths:
#     stats_fields = { 'mean': 'Slope Mean'}
#     gdf = gpd.read_file(output_segment_poly_path)
#     aggregate_raster_stats(DEM_path, gdf, output_shapefile_path=output_segment_poly_path, **stats_fields)
    #rank_elevation_mean(output_segment_poly_path, output_segment_poly_path)




In [37]:
import geopandas as gpd
from shapely.geometry import LineString, Polygon, Point
from skimage.morphology import skeletonize
from skimage import measure
import numpy as np
import rasterio

def get_centroid_elevation(centroid, dem):
    """
    Gets the elevation of a centroid by sampling the DEM.
    
    Parameters:
    centroid (shapely.geometry.Point): The centroid to get the elevation for.
    dem (rasterio.io.DatasetReader): The DEM raster data.
    
    Returns:
    elevation (float): The elevation at the centroid location.
    """
    coords = [(centroid.x, centroid.y)]
    for val in dem.sample(coords):
        return val[0]

def order_centroids_by_elevation(centroids, dem):
    """
    Orders centroids by elevation.
    
    Parameters:
    centroids (GeoDataFrame): The centroids to order.
    dem (rasterio.io.DatasetReader): The DEM raster data.
    
    Returns:
    ordered_centroids (GeoDataFrame): The centroids ordered by elevation.
    """
    # Get elevations for all centroids
    centroids['elevation'] = centroids.geometry.apply(lambda centroid: get_centroid_elevation(centroid, dem))
    
    # Sort by elevation in ascending order
    centroids = centroids.sort_values(by=['elevation'])
    
    return centroids

def create_centerline_from_skeleton(polygon, dem_path, buffer_size=10):
    """
    Creates a centerline from an irregular polygon representing a river channel
    using skeletonization. Connects adjacent centroids starting from the lowest
    elevation and going up to the highest.
    
    Parameters:
    polygon (shapely.geometry.Polygon): The input river channel polygon.
    dem_path (str): The file path to the DEM raster data.
    buffer_size (float): The buffer size used to find adjacent centroids.

    Returns:
    centerline (shapely.geometry.LineString): The generated centerline.
    centroids_gdf (geopandas.GeoDataFrame): The centroids as a GeoDataFrame for troubleshooting.
    """
    # Ensure the polygon is valid
    polygon = polygon.buffer(0)

    # Load the DEM
    dem = rasterio.open(dem_path)

    # Convert polygon to a binary image
    minx, miny, maxx, maxy = polygon.bounds
    width = int(np.ceil(maxx - minx))
    height = int(np.ceil(maxy - miny))
    
    # Create a grid of points within the polygon's bounding box
    x = np.linspace(minx, maxx, width)
    y = np.linspace(miny, maxy, height)
    x, y = np.meshgrid(x, y)
    
    # Create an image mask where points inside the polygon are 1, outside are 0
    mask = np.array([polygon.contains(Point(x[i, j], y[i, j])) for i in range(height) for j in range(width)])
    mask = mask.reshape((height, width))
    
    # Skeletonize the binary mask
    skeleton = skeletonize(mask)

    # Find the contours of the skeleton and create centroids
    contours = measure.find_contours(skeleton, 0.5)
    centroids = []
    for idx, contour in enumerate(contours):
        # Convert pixel coordinates back to original coordinates
        points = [(x[int(coord[0]), int(coord[1])], y[int(coord[0]), int(coord[1])]) for coord in contour]
        
        # Create a Polygon from the contour points (assuming triangles)
        if len(points) >= 3:
            triangle = Polygon(points)
            if triangle.is_valid and triangle.area > 0:
                centroid = triangle.centroid
                if polygon.contains(centroid):
                    centroids.append({'fid': idx, 'geometry': centroid})
    
    # Convert centroids to GeoDataFrame
    centroids_gdf = gpd.GeoDataFrame(centroids, crs="EPSG:6342")

    # Order centroids by elevation
    ordered_centroids_gdf = order_centroids_by_elevation(centroids_gdf, dem)

    # Create a LineString that connects the ordered centroids
    if len(ordered_centroids_gdf) > 1:
        centerline = LineString(ordered_centroids_gdf.geometry.tolist())
        # Ensure the centerline is fully within the original polygon
        if not polygon.contains(centerline):
            print("Warning: The generated centerline falls outside the polygon.")
           # centerline = None
    else:
        centerline = None  # Handle cases where no valid centerline could be generated

    return centerline, ordered_centroids_gdf

# Load your river channel polygon and DEM from a GeoPackage or other source
gdf = gpd.read_file(r"Y:\ATD\GIS\Bennett\Channel Polygons\ME_Channel.gpkg")
dem_path = r"Y:\ATD\GIS\Bennett\DEMs LIDAR\2013 DEM OT ndv.tif"

# Assuming there's only one polygon, or iterate through a list if there are multiple
polygon = gdf.geometry.iloc[0]
centerline, centroids_gdf = create_centerline_from_skeleton(polygon, dem_path, buffer_size=1)

# Save the centerline to a new layer in the GeoPackage
if centerline:
    centerline_gdf = gpd.GeoDataFrame(geometry=[centerline], crs=gdf.crs)
    centerline_gdf.to_file(r"Y:\ATD\GIS\Bennett\Channel Polygons\Centerlines\test_centerline.gpkg", driver="GPKG")
    print("Centerline generated successfully.")
else:
    print("No valid centerline could be generated.")

# Save the centroids to a new layer in the GeoPackage
centroids_gdf.to_file(r"Y:\ATD\GIS\Bennett\Channel Polygons\Centerlines\test_centroids.gpkg", driver="GPKG")
